In [1]:
import numpy as np

In [26]:
n = 300
m = 3*n
eps = 0.1
par_mu = 0
par_sd = np.sqrt(1)
out_mu = 5
out_sd = np.sqrt(1)

true_alpha = [par_mu, par_sd]

exper_iter = 1 #デバッグ後変える
mm_iter = 500
L = 1
learn_par = 1
dicay_par = 0.5

def sigmoid(x):
    return 1/(np.exp(-x) + 1)


def deriv_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))


def g_up(t, s):
    sigmoid(s) + deriv_sigmoid(s)*(t-s) + (t-s)^2/20

def g_lo(t, s):
    sigmoid(s) + deriv_sigmoid(s)*(t-s) - (t-s)^2/20


In [25]:
res = [0 for i in range(exper_iter)]
for i in range(exper_iter):
    print("%d/%d" %(i+1, exper_iter))
    data = np.random.normal(loc = par_mu, scale = par_sd, size = int(n*(1-eps)))
    contamination = np.random.normal(loc = par_mu, scale = par_sd, size = (n - int(n*(1-eps))))
    data = np.concatenate([data, contamination])
    np.random.shuffle(data)
    alpha = (np.mean(data), np.std(data)**2)
    for j in range(mm_iter):
        

1/1
